By : Fares Abbas 

In [4]:
# importing ( pandas , numpy , os , glob ) to read data from patch's
import pandas as pd 
import numpy as np 
import os
import glob as gb
import matplotlib.pyplot as plt 
import cv2

In [5]:
size = 100 
code = {'buildings':0 ,'forest':1,'glacier':2,'mountain':3,'sea':4,'street':5}

In [6]:
# read train data 
train_data = '../input/intel-image-classification/seg_train/seg_train/'

X_train = []
y_train = []
# open train data folder  ( os , glob )
for folder in os.listdir(train_data) :
    data = gb.glob(pathname= str(train_data + folder + '/*.jpg'))
    print(f'{len(data)} in folder {folder}')
    
    # read train images 
    for data in data : 
        images = cv2.imread(data)
        images_array = cv2.resize(images,(size,size))
        X_train.append(list(images_array))
        y_train.append(code[folder])

In [7]:
def getting_code(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x    

In [8]:
# show train images
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_train),49))) : 
    plt.subplot(7,7,n+1)
    plt.imshow(X_train[i])    
    plt.axis('off')
    plt.title(getting_code(y_train[i]))

In [9]:
# read test data 
test_data = '../input/intel-image-classification/seg_test/seg_test/'

X_test = []
y_test = []
# open test data folder  ( os , glob )
for folder in os.listdir(test_data) :
    data = gb.glob(pathname= str(test_data + folder + '/*.jpg'))
    print(f'{len(data)} in folder {folder}')
    # read test images 
    for data in data : 
        images = cv2.imread(data)
        images_array = cv2.resize(images,(size,size))
        X_test.append(list(images_array))
        y_test.append(code[folder])

In [10]:
# show test images
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_test),49))) : 
    plt.subplot(7,7,n+1)
    plt.imshow(X_test[i])    
    plt.axis('off')
    plt.title(getting_code(y_test[i]))

In [11]:
# read pred data 
pred_data = '../input/intel-image-classification/seg_pred/seg_pred/'


X_pred = [ ]
# open pred data folder  ( os , glob )
data = gb.glob(pathname= str(pred_data + '/*.jpg'))
print(f'{len(data)} ')
for data in data: 
    image = cv2.imread(data)
    image_array = cv2.resize(image , (size,size))
    X_pred.append(list(image_array))   

In [12]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_pred),49))) : 
    plt.subplot(7,7,n+1)
    plt.imshow(X_pred[i])    
    plt.axis('off')

In [13]:
# convert data to array 
X_train = np.array(X_train)
X_test = np.array(X_test)
X_pred_array = np.array(X_pred)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [14]:
print(f'X_train shape  is {X_train.shape}')
print(f'X_test shape  is {X_test.shape}')
print(f'X_pred shape  is {X_pred_array.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'y_test shape  is {y_test.shape}')

In [15]:
# Building CNN model 
import tensorflow as tf 
import keras

In [16]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(512,kernel_size=(3,3),activation='relu',input_shape=(size,size,3)),
        keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu'),
        keras.layers.Conv2D(256,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4), 
        keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'), 
        keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu'), 
        keras.layers.Conv2D(80,kernel_size=(3,3),activation='relu'),    
        keras.layers.MaxPool2D(4,4),
        keras.layers.Flatten() ,    
        keras.layers.Dense(512,activation='relu') ,       
        keras.layers.Dense(256,activation='relu') ,    
        keras.layers.Dense(128,activation='relu') ,    
        keras.layers.Dense(64,activation='relu') ,
        keras.layers.Dropout(rate=0.2) ,            
        keras.layers.Dense(6,activation='softmax') ,    
        ])

In [17]:
KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [18]:
print(KerasModel.summary())

In [19]:
epochs = 50
ThisModel = KerasModel.fit(X_train, y_train, epochs=epochs,batch_size=128,verbose=1)

In [20]:
ModelLoss, ModelAccuracy = KerasModel.evaluate(X_test, y_test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

In [21]:
y_pred = KerasModel.predict(X_test)

print('Prediction Shape is {}'.format(y_pred.shape))

In [24]:
y_result = KerasModel.predict(X_pred_array)
print('Prediction Shape is {}'.format(y_result.shape))

plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_pred),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_pred[i])    
    plt.axis('off')
    plt.title(getting_code(np.argmax(y_result[i])))